In [1]:
import os.path as osp
import os
import numpy as np
import pickle
import logging
import pickle
from collections import defaultdict,Counter
from tqdm import tqdm
import random
import cv2

In [7]:
import datetime

In [2]:
clsNum = 123
pickleDir = '/home/k100/Code/motionclassifiation_trainData_process/pyskl/data'
# picklePath = f'{pickleDir}/yolov8_stgcnpptrain4SHOW_20230915_cls_10.pkl'
picklePath = f'{pickleDir}/yolov8_stgcnpp_123_20241008.pkl'
save_path = '.'
save_data_pkl = 'test.pkl'
frames_drop_pkl = 'frames_drop_skes_test.pkl'

In [3]:
def one_hot_vector(labels):
    num_skes = len(labels)
    labels_vector = np.zeros((num_skes, clsNum))
    for idx, l in enumerate(labels):
        labels_vector[idx, l] = 1

    return labels_vector

def normalization(data,frameShape=(1080,1920)):
    height,width = frameShape
    data[...,0]/=width
    data[...,1]/=height
    return data


def split_dataset(skes_joints, labels, performer, camera, evaluation, save_path):
    print(performer.shape,camera.shape)
    train_indices, test_indices = get_indices(performer, camera, evaluation)
    m = 'sklearn'  # 'sklearn' or 'numpy'
    train_labels = labels[train_indices]
    test_labels = labels[test_indices]
    print('[train_indices] : ', len(train_indices))
    train_x = skes_joints[train_indices]
    train_y = one_hot_vector(train_labels)
    test_x = skes_joints[test_indices]
    test_y = one_hot_vector(test_labels)
    print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)
    save_name = save_path + f'/NTU60_{evaluation}.npz'
    np.savez(save_name, x_train=train_x, y_train=train_y, x_test=test_x, y_test=test_y)


def get_indices(performer, camera, evaluation='CS'):
    test_indices = np.empty(0)
    train_indices = np.empty(0)

    if evaluation == 'CS':  # Cross Subject (Subject IDs)
        train_ids = [1,  2,  4,  5,  8,  9,  13, 14, 15, 16,
                     17, 18, 19, 25, 27, 28, 31, 34, 35, 38]
        test_ids = [3,  6,  7,  10, 11, 12, 20, 21, 22, 23,
                    24, 26, 29, 30, 32, 33, 36, 37, 39, 40]

        # Get indices of test data
        for idx in test_ids:
            temp = np.where(performer == idx)[0]  # 0-based index
            test_indices = np.hstack((test_indices, temp)).astype(np.int)

        # Get indices of training data
        for train_id in train_ids:
            temp = np.where(performer == train_id)[0]  # 0-based index
            train_indices = np.hstack((train_indices, temp)).astype(np.int)
    else:  # Cross View (Camera IDs)
        train_ids = [2, 3]
        test_ids = 1
        # Get indices of test data
        temp = np.where(camera == test_ids)[0]  # 0-based index
        test_indices = np.hstack((test_indices, temp)).astype(np.int)

        # Get indices of training data
        for train_id in train_ids:
            temp = np.where(camera == train_id)[0]  # 0-based index
            train_indices = np.hstack((train_indices, temp)).astype(np.int)
    return train_indices, test_indices

def get_video_last_frame(vname):
    if not os.path.exists(vname):
        return None,None
    cap = cv2.VideoCapture(vname)
    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_FRAME_COUNT) - 1)
    ret, last_frame = cap.read()
    cap.release()

    return ret,last_frame

def xywh2xyxy(bbox):
    xc,yc,w,h = bbox
    x1 = xc - (w/2)
    y1 = yc - (h/2)
    x2 = xc + (w/2)
    y2 = yc + (h/2)
    return [x1,y1,x2,y2]

def xyxy2xywh(bbox):
    x1,y1,x2,y2 = bbox
    xc = (x1+x2)/2
    yc = (y1+y2)/2
    w = x2-x1
    h = y2-y1
    return [xc,yc,w,h]

def getBBOX_from_kpt(kpt,outsize=2):
    x1 = np.min(anno['keypoint'][:,-1,:,0])
    y1 = np.min(anno['keypoint'][:,-1,:,1])
    x2 = np.max(anno['keypoint'][:,-1,:,0])
    y2 = np.max(anno['keypoint'][:,-1,:,1])
    xc,yc,w,h = xyxy2xywh([x1,y1,x2,y2])
    w *= outsize
    h *= outsize
    x1,y1,x2,y2 = xywh2xyxy([xc,yc,w,h])
    bbox = np.array([x1,y1,x2,y2]).astype(int)
    bbox[bbox<0] = 0
    return bbox

In [4]:
with open(picklePath,'rb') as f:
    datas = pickle.load(f)
annotations = datas['annotations']
annotations = annotations#[:10]
# xsub_train = datas['split']['xsub_train']
# xsub_val = datas['split']['xsub_val']

In [5]:
nasPath = '/run/user/1001/gvfs/smb-share:server=10.173.1.14,share=shared_folders/stgcnpp_dataset/ntu_rgbd_120/videos'
saveRGBPath = '/home/k100/Code/motionclassifiation_trainData_process/RGB_lastFrame'

In [15]:
try:
    pbar.close()
except:
    pass
frames_cnt = np.zeros(len(annotations), dtype=np.int)
for annoIdx,anno in tqdm(enumerate(annotations)):
    frames_cnt[annoIdx] = anno['total_frames']
# max_num_frames = frames_cnt.max()  # 300
max_num_frames = 100

## =====================================================================

aligned_skes_joints = np.zeros((len(annotations), max_num_frames, 51*2), dtype=np.float32)
labels = []
# crops = []
crops_names = []
pbar = tqdm(total=len(annotations),desc = '[data no] : ')
count = 0
for annoIdx,anno in enumerate(annotations):
    pbar.update(1)
    frame_dir = anno['frame_dir']
    if 'rand' in frame_dir or '.txt' in frame_dir or '.png' in frame_dir or 'video_' in frame_dir:
        continue
    writeImgName = f'{saveRGBPath}/{frame_dir}_rgb.png'
    if os.path.exists(writeImgName):
        kpt = anno['keypoint']
        kptS = anno['keypoint_score']
        label = anno['label']
        img_shape = anno['img_shape']
        total_frames = anno['total_frames']
        personNum = len(kpt)
        frames_cnt[annoIdx] = total_frames
        kpt2D = kpt
        kpt = np.concatenate((kpt,np.expand_dims(kptS,-1)),axis=-1)#.shape
        kpt = normalization(kpt).reshape(total_frames,51*personNum)
        if total_frames > max_num_frames:
            kpt = kpt[-max_num_frames:]
            frames_cnt[annoIdx] = max_num_frames
        aligned_skes_joints[count, :kpt.shape[0],:kpt.shape[-1]] = kpt
        labels.append(label)
        crops_names.append(f'{frame_dir}_rgb.png')

        
        count += 1

        
        
        
    else:
        kpt = anno['keypoint']
        x1,y1,x2,y2 = getBBOX_from_kpt(anno['keypoint'])
        if x2-x1 < 64 or y2-y1 < 128:
            continue

        ret,lastFrame = get_video_last_frame(f'{nasPath}/{frame_dir}_rgb.avi')
        if not ret:
            print(f'[NOT return] : {nasPath}/{frame_dir}_rgb.avi')
            continue


        kptS = anno['keypoint_score']
        label = anno['label']
        img_shape = anno['img_shape']
        total_frames = anno['total_frames']
        personNum = len(kpt)
        frames_cnt[annoIdx] = total_frames
        kpt2D = kpt
        kpt = np.concatenate((kpt,np.expand_dims(kptS,-1)),axis=-1)#.shape
        kpt = normalization(kpt).reshape(total_frames,51*personNum)

        if total_frames > max_num_frames:
            kpt = kpt[-max_num_frames:]
            frames_cnt[annoIdx] = max_num_frames
        aligned_skes_joints[count, :kpt.shape[0],:kpt.shape[-1]] = kpt
        labels.append(label)
        cv2.imwrite(writeImgName,cv2.resize(lastFrame[y1:y2,x1:x2],(128,256)))
        crops_names.append(f'{frame_dir}_rgb.png')
        # crops.append(lastFrame[y1:y2,x1:x2])
        count += 1
pbar.close()

aligned_skes_joints.shape,len(labels),len(crops_names)

/tmp/ipykernel_780968/1031592507.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  frames_cnt = np.zeros(len(annotations), dtype=np.int)
1139105it [00:00, 2424781.28it/s]
[data no] : 100%|█████████████████████████████████████████████████████████████████████| 1139105/1139105 [00:10<00:00, 107874.31it/s]


((1139105, 100, 102), 215637, 215637)

In [16]:
aligned_skes_joints = aligned_skes_joints[:len(labels)]
aligned_skes_joints.shape,len(labels),len(crops_names)

((215637, 100, 102), 215637, 215637)

In [17]:
counter = Counter(labels)
min_key = min(counter, key=counter.get)
max_key = max(counter, key=counter.get)
max_key = max(counter, key=counter.get)
average = sum(counter.values()) / len(counter)
maxData_train = int(average)
maxData_test = int(average*0.1)
print(f'data min num class is {min_key} with num : {counter[min_key]}')
print(f'data max num class is {max_key} with num : {counter[max_key]}')
print(f'Average data num : {average}')
print(f'max train data num : {maxData_train}')
print(f'max test data num : {maxData_test}')

train_val_DataNumEachClass = defaultdict(lambda:[0,0])
for cls in counter:
    totalDataNumEachClass = int(counter[cls])
    trainDataNumEachClass = int(totalDataNumEachClass*0.9)
    trainDataNumEachClass = maxData_train if trainDataNumEachClass > maxData_train else trainDataNumEachClass
    valDataNumEachClass = totalDataNumEachClass - trainDataNumEachClass
    valDataNumEachClass = maxData_test if valDataNumEachClass > maxData_test else valDataNumEachClass
    train_val_DataNumEachClass[cls][0] = trainDataNumEachClass
    train_val_DataNumEachClass[cls][1] = valDataNumEachClass

data min num class is 41 with num : 422
data max num class is 0 with num : 2241
Average data num : 1796.975
max train data num : 1796
max test data num : 179


In [19]:
train_val_DataNumEachClassCurrent = defaultdict(lambda:[0,0])
new_datas = []
new_labels = []
new_crops = []
camera = []

pbar = tqdm(total=len(aligned_skes_joints),desc = '[data no] : ')
for dataIdx in range(len(aligned_skes_joints)):
    aligned_skes_joint = aligned_skes_joints[dataIdx]
    label = labels[dataIdx]
    crop = crops_names[dataIdx]
    pbar.update(1)
    trainCurrentNum = train_val_DataNumEachClassCurrent[label][0]
    valCurrentNum = train_val_DataNumEachClassCurrent[label][1]
    trainTargetNum = train_val_DataNumEachClass[label][0]
    valTargetNum = train_val_DataNumEachClass[label][1]
    if trainCurrentNum>=trainTargetNum and valCurrentNum>=valTargetNum:
        continue
    elif trainCurrentNum>=trainTargetNum and not valCurrentNum>=valTargetNum:
        new_datas.append(aligned_skes_joint)
        new_labels.append(label)
        new_crops.append(crop)
        camera.append(1)
        train_val_DataNumEachClassCurrent[label][1] += 1
    elif not trainCurrentNum>=trainTargetNum and valCurrentNum>=valTargetNum:
        new_datas.append(aligned_skes_joint)
        new_labels.append(label)
        new_crops.append(crop)

        camera.append(2)
        train_val_DataNumEachClassCurrent[label][0] += 1
    else:
        new_datas.append(aligned_skes_joint)
        new_labels.append(label)
        new_crops.append(crop)

        add2 = random.choice([1,2])
        if add2 == 2:
            camera.append(2)
            train_val_DataNumEachClassCurrent[label][0] += 1
        else:
            camera.append(1)
            train_val_DataNumEachClassCurrent[label][1] += 1
pbar.close()


[data no] : 100%|███████████████████████████████████████████████████████████████████████| 215637/215637 [00:00<00:00, 528236.54it/s]


In [20]:
counter = Counter(new_labels)
min_key = min(counter, key=counter.get)
max_key = max(counter, key=counter.get)
max_key = max(counter, key=counter.get)
average = sum(counter.values()) / len(counter)
print(f'data min num class is {min_key} with num : {counter[min_key]}')
print(f'data max num class is {max_key} with num : {counter[max_key]}')

data min num class is 41 with num : 422
data max num class is 0 with num : 1975


In [21]:
new_datas = np.array(new_datas)
camera = np.array(camera)
labels = np.array(new_labels)
crops = np.array(new_crops)

In [22]:
crops.shape,labels.shape,new_datas.shape

((214044,), (214044,), (214044, 100, 102))

In [23]:
rand_idxes = list(range(len(crops)))
random.shuffle(rand_idxes)
crops = crops[rand_idxes]
labels = labels[rand_idxes]
camera = camera[rand_idxes]
new_datas = new_datas[rand_idxes]

In [24]:
def split_dataset_withRGB(skes_joints, labels, performer, camera, evaluation, save_path, crops):
    print(performer.shape,camera.shape)
    train_indices, test_indices = get_indices(performer, camera, evaluation)
    m = 'sklearn'  # 'sklearn' or 'numpy'
    train_labels = labels[train_indices]
    test_labels = labels[test_indices]
    print('[train_indices] : ', len(train_indices))
    train_x = skes_joints[train_indices]
    test_x = skes_joints[test_indices]
    x_trainCrop = crops[train_indices]
    x_testCrop = crops[test_indices]
    
    train_y = one_hot_vector(train_labels)
    test_y = one_hot_vector(test_labels)
    print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)
    save_name = save_path + f'/NTU120_{evaluation}_RGB_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")}.npz'
    np.savez(save_name, x_train=train_x, y_train=train_y, x_test=test_x, y_test=test_y,
             x_train_crop = x_trainCrop,x_test_crop = x_testCrop)


In [25]:
evaluations = [ 'CV']#'CS'

performer = np.zeros(len(new_datas),dtype=int)
save_path = './testdata'
labels = np.array(labels)
for evaluation in evaluations:
    split_dataset_withRGB(aligned_skes_joints, labels, performer, camera, evaluation, save_path, crops)

(214044,) (214044,)
[train_indices] :  193302


/tmp/ipykernel_780968/4225601981.py:56: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_indices = np.hstack((test_indices, temp)).astype(np.int)
/tmp/ipykernel_780968/4225601981.py:61: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional informa

(193302, 100, 102) (193302, 123) (20742, 100, 102) (20742, 123)


In [27]:
np.unique(labels)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119])